In [24]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.1'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 338 kB in 3s (125 kB/s)
Reading package lists... Done


In [25]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [26]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [27]:
# Check schema
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [28]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView("home_sales")

In [29]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """(
  SELECT year(date) as Year, bedrooms as Bedrooms, format_number(avg(price), 2) as Avg_Price
  FROM home_sales
  WHERE bedrooms = 4
  GROUP BY 1, 2
  ORDER BY 1 ASC
)"""

spark.sql(query).show()


+----+--------+----------+
|Year|Bedrooms| Avg_Price|
+----+--------+----------+
|2019|       4|300,263.70|
|2020|       4|298,353.78|
|2021|       4|301,819.44|
|2022|       4|296,363.88|
+----+--------+----------+



In [30]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query="""(
  SELECT date_built as Year_Built, bedrooms as Bedrooms, bathrooms as Bathrooms, format_number(avg(price), 2) as Avg_Price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY 1, 2, 3
  ORDER BY 1 ASC
)"""

spark.sql(query).show()


+----------+--------+---------+----------+
|Year_Built|Bedrooms|Bathrooms| Avg_Price|
+----------+--------+---------+----------+
|      2010|       3|        3|292,859.62|
|      2011|       3|        3|291,117.47|
|      2012|       3|        3|293,683.19|
|      2013|       3|        3|295,962.27|
|      2014|       3|        3|290,852.27|
|      2015|       3|        3|288,770.30|
|      2016|       3|        3|290,555.07|
|      2017|       3|        3|292,676.79|
+----------+--------+---------+----------+



In [31]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query="""(
  SELECT date_built as Year_Built, bedrooms as Bedrooms, bathrooms as Bathrooms, floors as Floors,
    CASE WHEN sqft_living >= 2000 then '2000+' ELSE '<2000' END AS SqFt_Living, format_number(avg(price), 2) as Avg_Price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY 1, 2, 3, 4, 5
  ORDER BY 1 ASC
)"""

spark.sql(query).show()

+----------+--------+---------+------+-----------+----------+
|Year_Built|Bedrooms|Bathrooms|Floors|SqFt_Living| Avg_Price|
+----------+--------+---------+------+-----------+----------+
|      2010|       3|        3|     2|      2000+|285,010.22|
|      2011|       3|        3|     2|      2000+|276,553.81|
|      2012|       3|        3|     2|      2000+|307,539.97|
|      2013|       3|        3|     2|      2000+|303,676.79|
|      2014|       3|        3|     2|      2000+|298,264.72|
|      2015|       3|        3|     2|      2000+|297,609.97|
|      2016|       3|        3|     2|      2000+|293,965.10|
|      2017|       3|        3|     2|      2000+|280,317.58|
+----------+--------+---------+------+-----------+----------+



In [33]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query="""(
  SELECT view as View, round(avg(price), 2) as Avg_Price
  FROM home_sales
  WHERE price >= 350000
  GROUP BY 1
  ORDER BY 2 DESC
)"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|View| Avg_Price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 0.48079681396484375 seconds ---


In [34]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [35]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [37]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
query="""(
  SELECT view as View, round(avg(price), 2) as Avg_Price
  FROM home_sales
  WHERE price >= 350000
  GROUP BY 1
  ORDER BY 2 DESC
)"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|View| Avg_Price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 0.31448888778686523 seconds ---


In [38]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("parquet_partitioned_home_sales")

In [39]:
# 11. Read the parquet formatted data.
p_home_sales_p_df=spark.read.parquet("parquet_partitioned_home_sales")

In [40]:
# 12. Create a temporary table for the parquet data.
p_home_sales_p_df.createOrReplaceTempView("p_home_sales_p")

In [42]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
query="""(
  SELECT view as View, round(avg(price), 2) as Avg_Price
  FROM p_home_sales_p
  WHERE price >= 350000
  GROUP BY 1
  ORDER BY 2 DESC
)"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|View| Avg_Price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 0.45612597465515137 seconds ---


In [43]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [44]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False